In [151]:
import json
from pathlib import Path
from pprint import pprint

In [152]:
ds_path = Path.cwd() / "capital.json"
dataset = json.loads(ds_path.read_text())

In [153]:
for sample in dataset:
    if "tag" in sample:
        if "tags" in sample:
            sample["tags"].extend(sample["tag"])
        else:
            sample["tags"] = sample["tag"]
        del sample["tag"]

    sample["tags"] = set(sample["tags"])
    if sample["input"].lower() == sample["input"]:
        sample["tags"].add("all_lower")
        if "all_upper" in sample["tags"]:
            sample["tags"].remove("all_upper")
    if sample["input"].upper() == sample["input"]:
        sample["tags"].add("all_upper")
        if "all_lower" in sample["tags"]:
            sample["tags"].remove("all_lower")

    if sample["input"][0].isupper():
        sample["tags"].add("start_cap")
        if "start_lower" in sample["tags"]:
            sample["tags"].remove("start_lower")
    if sample["input"][0].islower():
        sample["tags"].add("start_lower")
        if "start_cap" in sample["tags"]:
            sample["tags"].remove("start_cap")

    if "shared_samples" in sample["tags"]:
        sample["tags"].remove("shared_samples")

    if sample["input"].split(' ')[0].lower() == "the":
        sample["tags"].add("start_the")
    if sample["input"].split(' ')[0].lower() != "the" and "start_the" in sample["tags"]:
        sample["tags"].remove("start_the")

    if len(sample["input"].split(' ')) % 2 == 0:
        sample["tags"].add("even_words")
        if "odd_words" in sample["tags"]:
            sample["tags"].remove("odd_words")
    if len(sample["input"].split(' ')) % 2 == 1:
        sample["tags"].add("odd_words")
        if "even_words" in sample["tags"]:
            sample["tags"].remove("even_words")

    if "fragment_verb" in sample["tags"] or "fragment_subject" in sample["tags"]:
        if "sentence" in sample["tags"]:
            sample["tags"].remove("sentence")

    sample["tags"] = list(sample["tags"])

input_duplicates = {}
for i, sample in enumerate(dataset):
    if sample["input"].lower() in input_duplicates:
        input_duplicates[sample["input"].lower()].append(i)
    else:
        input_duplicates[sample["input"].lower()] = [i]
for input, indices in input_duplicates.items():
    if len(indices) > 1:
        for i in indices:
            dataset[i]["tags"].append("shared_sample")


for sample in dataset:
    sample["tags"] = set(sample["tags"])
    sample["tags"].add("gpt4")
    sample["tags"] = list(set(sample["tags"]))

In [154]:
proper_indices = [i for i, sample in enumerate(dataset) if "proper_noun" in sample["tags"]]

In [155]:
json.dump(dataset, ds_path.open("w"), indent=2)
pprint(dataset)
print(f"Dataset length: {len(dataset)}")

[{'input': 'THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
           'nature',
           'odd_words',
           'start_the',
           'gpt4']},
 {'input': 'TODAY IS A BEAUTIFUL DAY.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
           'odd_words',
           'gpt4']},
 {'input': 'I LOVE TO TRAVEL AND EXPLORE NEW PLACES.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
           'even_words',
           'gpt4']},
 {'input': 'PYTHON IS A POPULAR PROGRAMMING LANGUAGE.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
           'even_words',
           'gpt4',
           'proper_noun']},
 {'input': 'THE SUN SETS IN THE WEST.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
  

In [157]:
tags = {}
for sample in dataset:
    for tag in sample["tags"]:
        if tag in tags:
            tags[tag] += 1
        else:
            tags[tag] = 1
print(len(dataset))
pprint(tags)

200
{'all_lower': 67,
 'all_upper': 59,
 'city': 25,
 'even_words': 97,
 'fragment_subject': 20,
 'fragment_verb': 20,
 'gpt4': 200,
 'nature': 62,
 'odd_words': 103,
 'proper_noun': 32,
 'sentence': 160,
 'shared_sample': 42,
 'start_cap': 133,
 'start_lower': 67,
 'start_the': 57}
